In [1]:
import constants
cache_folder_name = 'ratio=logreg;loss=squared;method=pymanopt_UB'
seed = 44

In [2]:
import autograd.numpy as np
np.random.seed(seed)
import python_utils.python_utils.basic as basic
import python_utils.python_utils.caching as caching
log_folder = '%s/%s' % (constants.cache_folder, cache_folder_name)
caching.init(constants.cache_folder, constants.which_hash, basic.archiver(log_folder))
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('Agg')

In [3]:
constants.cache_folder

'/Users/fultonw/Documents/projects/runs/domain_adapt_run/domain_adapt_run/notebooks/new/hyperparameter_loss_surface_stratified/better_v_data/cache'

In [4]:
import matplotlib.pyplot as plt
from IPython.display import display_pretty, display_html
import python_utils.python_utils.basic as basic
import pandas as pd
import pdb
import itertools
import functools
import scipy.stats
import scipy
import cvxopt
import autograd
import autograd.numpy as np
import domain_adapt.domain_adapt.new.fxns as fxns
import domain_adapt.domain_adapt.new.utils as utils
import domain_adapt.domain_adapt.new.optimizers as optimizers
import domain_adapt.domain_adapt.new.constructors as constructors
import domain_adapt.domain_adapt.SDR_data as data
import domain_adapt_run.domain_adapt_run.notebooks.new.hyperparameter_loss_surface_stratified.run as run
#import data.domain_adapt_data.domain_adapt_data.fxns as data

Couldn't import dot_parser, loading of dot files will not be possible.


In [5]:
# define data
x_dim = 50
num_train = 50
num_test = 50
proportion = 0.5
def get_data(seed):
    np.random.seed(seed)
    return data.better_v_data(x_dim=x_dim, num_train=num_train, num_test=num_test, proportion=proportion)

In [6]:
# define fitter
pymanopt_options={'logverbosity':2, 'maxiter':100}
#u_dim = 1
unconstrained = False
num_tries = 5
plot_b_info = constructors.plot_b_info
def get_fitter(c_lsqr_loss, c_lsqr_loss_eval, c_lsqr, c_logreg, weight_reg, UB_reg, sigma, max_ratio, u_dim):
    B_init_f_getter = lambda xs_train, ys_train, xs_test: (lambda: np.random.normal(size=(xs_train.shape[1],u_dim)))
    return constructors.logreg_ratio_UB_fitter(c_lsqr, c_logreg, weight_reg, UB_reg, sigma, B_init_f_getter, unconstrained, c_lsqr_loss, c_lsqr_loss_eval, max_ratio=max_ratio, num_tries=num_tries, pymanopt_options=pymanopt_options, plot_b_info=plot_b_info)

In [7]:
# define hyperparameter search space via an iterator
UB_regs = [0.]#1,1.,10.]
u_dims = [1,]
c_lsqr_losses = [1.]
c_lsqr_loss_evals = [0.]
c_lsqrs = [.1,1.,10.,100]
c_logregs = [0.1,1.,10.]
weight_regs = [0.1,1.,10.]
sigmas = [0.25,1.,5.]#,5.]
max_ratios = [5.]
make_tuple = lambda key, vals: [(key,val) for val in vals]
care_hyperparam_tuples = itertools.product(
    make_tuple('UB_reg',UB_regs), 
    make_tuple('u_dim',u_dims), 
    )
nocare_hyperparam_tuples = itertools.product(
    make_tuple('c_lsqr_loss',c_lsqr_losses), 
    make_tuple('c_lsqr_loss_eval',c_lsqr_loss_evals), 
    make_tuple('c_lsqr',c_lsqrs), 
    make_tuple('c_logreg',c_logregs), 
    make_tuple('weight_reg',weight_regs), 
    make_tuple('UB_reg',UB_regs), 
    make_tuple('sigma',sigmas), 
    make_tuple('max_ratio',max_ratios)
    )


In [8]:
#
mapper = map
#mapper = functools.partial(basic.joblib_parallel_map, 2, True)

In [ ]:
# 
num_iterations = 5

In [ ]:
# 
run.run(get_data, get_fitter, care_hyperparam_tuples, nocare_hyperparam_tuples, num_iterations, mapper)